### Train model


In [ ]:
import torch
from torch.utils.data import DataLoader 
import numpy as np
import matplotlib.pyplot as plt
import os
from torchmetrics import FBetaScore, Accuracy
import torchvision.transforms as T
from torchvision.transforms import functional as F

import src.dfmaker as dfmaker
import src.BWDatasets as BWDatasets

from paddleocr import PaddleOCR
import paddle


In [ ]:
seed = 115
generator = torch.Generator()
generator.manual_seed(seed)
np_generator = np.random.default_rng(seed=seed)

In [ ]:
device = torch.device('cpu')

# Make sure the notebook is deterministic if training on gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False

print(f'Cuda availability is: {torch.cuda.is_available()}')  # Returns True if a GPU is available
print(f"Training on device {device}.")